# *QuantNbody* tutorials : spin operators and fermionic Hamiltonians 

**Dr. Saad Yalouz**

**Laboratoire de Chimie Quantique de Strasbourg, France**

**July 2022**

In this QuantNBody tutorial, we will focus on the construction of spin operators like $S_2$ and $S_z$ and ab initio electronic structure Hamiltonians $H$. We will show how easily we can build these operators and use them practically.

Let us first import our package and some additional libraries

In [15]:
import quantnbody as qnb
import numpy as np
import scipy
import pyscf

## Electronic strcuture Hamiltonian

The ab initio electronic structure Hamiltonian describes how a collection of $N_e$ electrons rearrange around a set of positively charged nuclei in a molecule. This operator is defined like:

$$ H = \sum_{p,q} h_{pq} \hat{E}_{pq} + \frac{1}{2} \sum_{pqrs} g_{pqrs} \hat{e}_{pqrs} + \mathcal{E}_{nuc.\ rep.}$$

where we have the one- and two-electron integrals $h_{pq}$ and $g_{pqrs}$ (and the nuclear repulsion energy $\mathcal{E}_{nuc.\ rep.}$) associated to the one- and two-electron spin-free excitation operators defined like

$$ E_{pq} = \sum_\sigma a^\dagger_{p,\sigma}a_{q,\sigma} $$

and 

$$ e_{pqrs} = E_{pq}E_{rs} - \delta_{rs} E_{ps} .$$

Here again, we see that all these operators can be defined in terms of the $a_{p,\sigma}^\dagger a_{q,\tau}$ operators. 

Below we show how to create such Hamiltonian for the $H_4$ molecule in the STO-3G basis, and how to compute a ground state potential energy surface easily. In this case, we need to get access to the electronci integrals of the system. We use for this PySCF but note that Psi4 could have been used too !

First we build the essential many-body basis and a_Dagger_a operators

In [18]:
N_elec = N_MO = 4 # Number of MOs and electrons in the system

nbody_basis = qnb.fermionic.tools.build_nbody_basis( N_MO, N_elec ) 
a_dagger_a  = qnb.fermionic.tools.build_operator_a_dagger_a( nbody_basis )  

Then we can build the PES of the molecule with a loop

In [11]:
list_theta = np.linspace(num = 30, start=20. * np.pi/180., stop=160. * np.pi/180., endpoint=True) 
E_HF  = []
E_FCI = []
E_FCI_me = []

dim_parameter = len(list_theta)
Energies_FCI    = np.zeros((dim_parameter,MAX_ROOT))
Energies_FCI_me = np.zeros((dim_parameter,dim_H)) 

elem = 0
for theta in tqdm(list_theta, file=sys.stdout): 
    r = 1.
     
    XYZ_geometry = """ H   {0}   {1}  0.
                       H   {0}  -{1}  0. 
                       H  -{0}   {1}  0.
                       H  -{0}  -{1}  0.  """.format( r*np.cos(theta/2.), r*np.sin(theta/2.) ) 
     
    mol = gto.Mole()
    mol.build( atom     = XYZ_geometry,  # in Angstrom
               basis    = 'STO-3G',
               symmetry = False,
               spin     = 0 )

    mf = scf.RHF(mol)
    print('      ', end='')
    mf.kernel() 
    FCI_solver = pyscf.fci.FCI( mf, mf.mo_coeff  )  
    FCI_solver = fci.addons.fix_spin_(FCI_solver, ss=0, shift=10 ) 
    
    FCI_solver.nroots = MAX_ROOT
    E_FCI_pyscf, Wfn_FCI = FCI_solver.kernel()           
    h_MO = np.einsum('pi,pq,qj->ij', mf.mo_coeff,
                           mol.intor_symmetric('int1e_kin') + mol.intor_symmetric('int1e_nuc'),
                           mf.mo_coeff) 
    g_MO  = ao2mo.kernel( mol, mf.mo_coeff )
    g_MO  = ao2mo.restore('s1', g_MO, N_MO)
 

    E_rep_nuc = mol.energy_nuc()                  # Nuclei repuslion energy
 
    mol_qnb.build_hamiltonian_quantum_chemistry(h_MO, g_MO)  
    eig_energies, eig_vectors =  np.linalg.eigh(mol_qnb.H.A) 
    E_FCI_me += [ E_rep_nuc + eig_energies[0] ]
    
    Energies_FCI_me[elem,:] = [ E_rep_nuc + eig_energies[p]  for p in range(dim_H) ] 
    
    Energies_FCI[elem,:] = E_FCI_pyscf
    
    elem += 1 

AttributeError: module 'quantnbody' has no attribute 'Hamiltonian'

## Applying excitations to a state   

In this final part we show the effect of applying excitations to a reference wavefunction. For this, we will consider implementing a singlet excitation over an initial configuration to produce the final state

$$ | \Psi \rangle = (a^\dagger_{3,\alpha}a_{2,\alpha} + a^\dagger_{3,\beta}a_{2,\beta})| 11110000\rangle / \sqrt{2} $$

This is very easy to implement with the QuantNBody package. In this case, as shown below, the second quantization algebra can be very straightforwardly implemented in a few line of python code !

In [8]:
# We first translate the occupation number config into the many-body basis of kappa vectors
initial_config_occ_number = [ 1, 1, 1, 1, 0, 0, 0, 0 ]
initial_config = qnb.fermionic.tools.my_state( initial_config_occ_number, nbody_basis)

# Then we build the excitation operator
Excitation_op = (a_dagger_a[4,2] + a_dagger_a[5,3]) / np.sqrt(2)

# We apply the excitation on the intial state and store it into a Psi WFT
Psi = Excitation_op  @ initial_config

# We visualize the final wavefunction
qnb.fermionic.tools.visualize_wft(Psi,nbody_basis)
print()


	-----------
	 Coeff.      N-body state
	-------     -------------
	-0.70711	|11011000⟩
	+0.70711	|11100100⟩


